# Riiid! Answer Correctness Prediction
## Introduction
在本次比賽中，您將預測每個學生能夠正確回答的問題。 您將遍歷一系列問題。 做出預測後，您可以繼續進行下一批。

本次比賽與大多數 Kaggle 比賽的不同之處在於：
* 您只能從 Kaggle Notebooks 提交
* 您必須使用我們自定義的 **`riiiducation`** Python 模塊。 該模塊的目的是控制信息流，以確保您不會使用未來數據進行預測。 如果您沒有正確使用此模塊，您的代碼可能會失敗。

## 在本入門筆記本中，我們將展示如何使用 **`riiiducation`** 模塊來獲取測試功能並進行預測。
## TL;DR：端到端使用示例
```
import riiideducation
env = riiideducation.make_env()

# 訓練數據照常在比賽數據集中訓練數據照常在比賽數據集中
train_df = pd.read_csv('/kaggle/input/riiideducation/train.csv', low_memory=False)
train_my_model(train_df)
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df['answered_correctly'] = 0.5
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])```
請注意，`train_my_model` 是您需要為上述示例工作而編寫的函數。

## 深入介紹
首先讓我們導入模塊並創建一個環境。

In [ ]:
import riiideducation
import pandas as pd

# You can only call make_env() once, so don't lose it!
env = riiideducation.make_env()

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 訓練數據照常在比賽數據集中
它比默認設置的內存要大，所以我們將指定更有效的數據類型，現在只加載數據的子集。

In [ ]:
train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', low_memory=False, nrows=10**5, 
                       dtype={'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 'content_type_id': 'int8',
                              'task_container_id': 'int16', 'user_answer': 'int8', 'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32', 
                             'prior_question_had_explanation': 'boolean',
                             }
                      )
train_df

## `iter_test` 函數

循環遍歷測試集中每批問題的生成器。 為方便起見，您可以直接訪問示例測試行，但您的代碼只能通過 API 從真實測試集中獲取行。 調用 **`predict`** 後，您可以繼續下一批。

產量：
* 雖然有更多批次，並且自上次產量以來成功調用了 `predict`，但會產生一個元組：
     * `test_df`：具有下一批測試功能的 DataFrame，以及上一批的用戶響應。
     * `sample_prediction_df`：帶有示例預測的 DataFrame。 旨在填寫並傳遞回“預測”函數。
* 如果 `predict` 自上次 yield 後未成功調用，則打印錯誤並產生 `None`。

In [ ]:
# You can only iterate through a result from `env.iter_test()` once
# so be careful not to lose it once you start iterating.
iter_test = env.iter_test()

讓我們獲取第一個測試批次的數據並檢查一下。

In [ ]:
(test_df, sample_prediction_df) = next(iter_test)
test_df

In [ ]:
sample_prediction_df

請注意，如果我們嘗試繼續下一批而不對當前批次進行預測，我們將得到一個錯誤。

In [ ]:
next(iter_test)

### **`預測`** 函數
存儲您對當前批次的預測。 期望與您在從 `iter_test` 生成器返回的 `sample_prediction_df` 中看到的格式相同。

參數：
* `predictions_df`：必須與`sample_prediction_df`具有相同格式的DataFrame。

如果在 `iter_test` 生成器成功迭代後未調用此函數，則會引發異常。

讓我們使用 `iter_test` 提供的樣本進行虛擬預測。

In [ ]:
env.predict(sample_prediction_df)

## 主循環
讓我們遍歷測試集生成器中的所有剩餘批次，並對每個批次進行默認預測。 一旦你到達終點，`iter_test` 生成器將簡單地停止返回值。

在編寫自己的 Notebooks 時，一定要編寫健壯的代碼，盡可能少地對 `iter_test`/`predict` 循環做出假設。 例如，測試集包含以前在訓練中未觀察到的問題 ID。

您可以假設 `sample_prediction_df` 的結構在本次比賽中不會改變。

**不應提交 `test_df` 中的講座行。**

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df['answered_correctly'] = 0.5
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

## 重新啟動筆記本以再次運行您的代碼
為了打擊作弊，您只能在每次 Notebook 運行時調用 `make_env` 或迭代 `iter_test` 一次。 然而，當你在你的模型上迭代時，嘗試一些東西是合理的，稍微改變一下模型，然後再試一次。 不幸的是，如果您嘗試簡單地重新運行代碼，甚至刷新瀏覽器頁面，您仍然會在之前運行的同一 Notebook 執行會話上運行，並且 `riideducation` 模塊仍然會拋出錯誤。 要解決此問題，您需要明確重新啟動 Notebook 執行會話，您可以通過**單擊 Notebook Editor 頂部菜單欄中的“Run”->“Restart Session”** 來完成。## Restart the Notebook to run your code again
In order to combat cheating, you are only allowed to call `make_env` or iterate through `iter_test` once per Notebook run.  However, while you're iterating on your model it's reasonable to try something out, change the model a bit, and try it again.  Unfortunately, if you try to simply re-run the code, or even refresh the browser page, you'll still be running on the same Notebook execution session you had been running before, and the `riideducation` module will still throw errors.  To get around this, you need to explicitly restart your Notebook execution session, which you can do by **clicking "Run"->"Restart Session"** in the Notebook Editor's menu bar at the top.